Bibliotecas necessárias para o projeto

Ambiente virtual instalação:
1. pip install virtualenv
2. pip install virtualenvwrapper-win

Criação do ambiente:
1. python -m venv <nome do ambiente virtual>

Manipulação:
Abrir ambiente virtual -> .\coderhouse\Scripts\Activate.ps1
Desativar -> deactivate

In [8]:
import pandas as pd, requests as rq, sqlite3, datetime as dt, pprint 
from plyer import notification

**\# Função de alerta**

In [9]:
def alerta(nivel,base,etapa):
    """
   Function para gerar alertas de erros

    :param nivel: int
    :param base: string
    :param etapa: string
    :return notificacao
    
    """ 
    msg = ''
    match nivel:
        case 1:
            msg = 'Falha ao inserir dados no banco'
        case 2:
            msg = 'Falha ao carregar a API'
        case 3:
            msg = 'Falha ao ler dados no banco'
        case _:
            msg= 'Ocorreu um erro verifique!!'

    notificacao =  notification.notify(
                    title = msg,
                    message = f'{msg} {base}, na etapa: {etapa}',
                    app_name = etapa,     
                    timeout = 10,
             )

    return notificacao

**\# Banco de dados**

In [10]:
def inserir_dadosDB(df, nome_tabela):
    """
   Function para inserir um dataframe no banco de dados

    :param df: dataframe
    :param nome_tabela: string
    
    """ 

    conn = sqlite3.connect('coderhouse.db')

    # Escrever o DataFrame na tabela 'nome_tabela'
    df.to_sql(nome_tabela, conn, if_exists='replace', index=False)

    conn.close()

    return True

def ler_dadosDB(nome_tabela):
    """
        Function para ler uma tabela no banco e retornar um dataframe

        :param nome_tabela: string
        :return df: dataframe
    
    """ 
    
    con = sqlite3.connect("coderhouse.db")

    query = f"SELECT * FROM {nome_tabela}"
    df = pd.read_sql(query,con)

    con.close()

    return df


def sys_tables():
    """
        Function que retorna um dataframe com as tabelas no banco de dados
        :return schema: dataframe
    """

    con = sqlite3.connect("coderhouse.db")
    query = f"SELECT name from sqlite_master WHERE type = 'table'"
    schema = pd.read_sql_query(query, con)
    con.close()
    return schema


**Extração**

In [11]:
# cnpj = '54517628000198'

def  tipo_url(tipo, dado):
    """
        Function que retorna o end point de acordo com os parametros passados
        :param tipo: string
        :param dado: string
        :return url: string
    """    
    tipo = tipo.lower()
    if tipo == 'cnpj':
        url  = f'https://brasilapi.com.br/api/cnpj/v1/{dado}'
    elif tipo == 'clima':
        url  = f'https://brasilapi.com.br/api/cptec/v1/clima/previsao/{dado}'
    elif tipo == 'cidade':
        url  = f'https://brasilapi.com.br/api/cptec/v1/cidade/{dado}'
    else: 
        raise Exception(f"erro request {tipo} ou {dado}invalido")
    return url
  

def get_api(dado):
    """
        Function que retorna os dados do end point
        :param dado: string
        :return data_jason: dataframe
    """       
    url = dado
    response =  rq.get(url)
    if response.status_code == 200:
        data_jason = response.json()
    else:
        alerta(1,'cnpj', 'Falha ao carregar dados do cnpj') #// TODO ajustar mensagem
    
    return data_jason



**Transformação**

In [12]:
# pprint.pprint(data_jason)    

def trata_dataframe(tipo,dataframe):
    """
        Function que retorna um dataframe com os dados tratados de acordo com os dados passados
        :param tipo: string
        :param dataframe: dataframe
        :return dados: dataframe
    """    
    if tipo == 'empresa':
        
        data_json = dataframe
        cnpj = [data_json['cnpj'] for item in data_json]
        bairros = [data_json['bairro'] for item in data_json]
        razao_social = [data_json['razao_social'] for item in data_json]
        capital_social = [data_json['capital_social'] for item in data_json]
        cep = [data_json['cep'] for item in data_json]
        cnae_fiscal = [data_json['cnae_fiscal'] for item in data_json]
        cnae_fiscal_descricao = [data_json['cnae_fiscal_descricao'] for item in data_json]
        cnae_fiscal_descricao = [data_json['cnae_fiscal_descricao'] for item in data_json]
        codigo_municipio = [data_json['codigo_municipio'] for item in data_json]
        municipio = [data_json['municipio'] for item in data_json]

        dado = pd.DataFrame({
        'cnpj': cnpj,
        'razao_social': razao_social,
        'capital_social': capital_social,
        'cnae_fiscal': cnae_fiscal,
        'cnae_fiscal_descricao': cnae_fiscal_descricao,
        'bairro': bairros,
        'cep': cep,
        "codigo_municipio": codigo_municipio,
        "municipio": municipio,
        })
        dados = dado.head(1)

    elif tipo == 'cidade':
        data_json = dataframe
        # cityName = data_json['municipio'].to_string(index=False)
        nome = [data_json[0]['nome']for item in data_json]
        id_estado = [data_json[0]['id'] for item in data_json]
        estado = [data_json[0]['estado'] for item in data_json]

        dado = pd.DataFrame({
        'nome': nome,
        'id': id_estado,
        'estado': estado,
        })
        
        dados = dado.head(1)

    elif tipo == 'clima':
        # cityName = '3030'
        # cityCode = descr_cidades['id'].to_string(index=False)
        data_json = dataframe
        cidade = [data_json['cidade']for item in data_json]
        estado = [data_json['estado'] for item in data_json]
        clima = [data_json['clima'][0]['condicao_desc'] for item in data_json]
        temperatura_max = [data_json['clima'][0]['max'] for item in data_json]
        temperatura_min = [data_json['clima'][0]['min'] for item in data_json]
        atualizado_em = [data_json['atualizado_em'] for item in data_json]


        dado = pd.DataFrame({
            'cidade': cidade,
            'estado': estado,
            'clima': clima,
            'temperatura_max': temperatura_max,
            'temperatura_min': temperatura_min,
            'atualizado_em': atualizado_em,
        })
        dados = dado.head(1)
    else:
        raise Exception(f"erro request {dataframe} invalido")
    return dados

**Main**

In [13]:
def main(cnpj):
    """
        Function principal que faz a chamada de todas as funções
        :param cnpj: string
    """    
    #CNPJ

    url = tipo_url('cnpj', cnpj)

    data_empresa = get_api(url)
    empresa = trata_dataframe('empresa',data_empresa)

    #CIDADE

    cidade = empresa['municipio'].to_string(index=False)
    url = tipo_url('cidade', cidade)
    data_cidade = get_api(url)
    cidades = trata_dataframe('cidade',data_cidade)

    #CLIMA

    clima = cidades['id'].to_string(index=False)
    url = tipo_url('clima', clima)

 
    data_clima = get_api(url)
    climas = trata_dataframe('clima',data_clima)
 
    descr_empresa = [empresa,climas]
    result = pd.concat(descr_empresa, axis=1)
    inserir_dadosDB(result,'empresa')


**LOAD**

In [14]:
cnpj = input('Digite o cnpj da empresa que deseja fazer uma consulta') #06990590000123

main(cnpj)


# sys_tables()
df = ler_dadosDB('empresa')
df.head()

,cnpj,razao_social,capital_social,cnae_fiscal,cnae_fiscal_descricao,bairro,cep,codigo_municipio,municipio,cidade,estado,clima,temperatura_max,temperatura_min,atualizado_em
0,06990590000123,GOOGLE BRASIL INTERNET LTDA.,200000000,6319400,"Portais, provedores de conteúdo e outros servi...",ITAIM BIBI,04538133,7107,SAO PAULO,São Paulo,SP,Parcialmente Nublado,20,16,2023-10-18


In [15]:
df.shape

(1, 15)